In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Директория, где располагаются данные

In [2]:
data_dir = '/content/drive/My Drive/Colab Notebooks/Yandex/resources/'

## Анализ логов продуктового магазина

Представьте, что вы работаете аналитиком онлайн-магазина и ваша цель - повысить GMV (суммарную выручку от продажи товаров). Для этого вы хотите провести ряд исследований и придумать какие-то меры для увеличения выручки.

В файле <i>orders_with_items.json</i> представлена информация о заказах в онлайн-магазине.
```
OrderID - ID Заказа,
UserID - ID клиента,
OrderTime - время заказа,
items - список элементов в заказе, каждый элемент описан словарем:
    ItemID  - ID продукта, 
    price - цена единицы товара, 
    quantity - количество единиц товара в заказе
```
В файле <i>items_with_categ.csv</i> представлена информация о категории товаров.
```
ItemID - ID продукта,
Category - категория продукта,
ParentCategory - более крупная категория продукта
```

1. Для начала поймем, какого числа было сделано больше всего заказов?
2. Давайте посмотрим на категории товаров. Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)?
3. Представьте, что мы хотим запустить какую-то акцию для клиентов или предложить какую-то бонусную программу. Какую акцию было бы целесообразно провести и на каких клиентов её ориентировать? Составьте какой-то список клиентов, которым вы бы предложили участие в акции/программе.
4. Представьте, что для повышения GMV мы хотим внедрить на сайте систему рекомендаций и предлагать пользователю какие-то товары по мере сбора корзины. Что бы вы порекомендовали пользователю X?

Если для решения задачи вам потребуются какие-то сторонние библиотеки, вы можете их установить. __[Документация по установке пакетов](https://cloud.yandex.ru/docs/datasphere/operations/projects/install-dependencies)__

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

1. Для начала поймем, какого числа было сделано больше всего заказов?

In [4]:
owi = pd.read_json('/content/drive/My Drive/Colab Notebooks/Yandex/resources/orders_with_items.json')
owi.head()

,OrderID,UserID,OrderTime,items
0,90659865,1.609460e+18,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit..."
1,90622054,2.499293e+18,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit..."
2,90659952,3.945866e+17,2021-01-01 05:48:53,"[{'ItemID': '306553', 'price': 101.9, 'quantit..."
3,89765281,5.412765e+18,2021-01-01 08:33:41,"[{'ItemID': '979703', 'price': 99.9, 'quantity..."
4,90582308,6.380061e+18,2021-01-01 09:55:35,"[{'ItemID': '303765', 'price': 44.9, 'quantity..."


In [5]:
owi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40375 entries, 0 to 40374
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   OrderID    40375 non-null  int64  
 1   UserID     40375 non-null  float64
 2   OrderTime  40375 non-null  object 
 3   items      40375 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.2+ MB


In [6]:
owi.columns

Index(['OrderID', 'UserID', 'OrderTime', 'items'], dtype='object')

Чтобы решить, какого числа было сделано наибольшее количество заказов, нужно сгруппировать данные по дате заказа, посчитать количество заказов по каждой дате, найти максимальное количество заказов и определить дату.

У нас дата находится в колонке ['OrderTime'], сейчас в этой колонке тип данных object. Переведём данные в тип datetime64

In [7]:
import datetime

In [8]:
pd.to_datetime(owi.OrderTime)

0       2021-01-01 03:51:42
1       2021-01-01 04:13:13
2       2021-01-01 05:48:53
3       2021-01-01 08:33:41
4       2021-01-01 09:55:35
                ...        
40370   2021-06-30 23:40:23
40371   2021-06-30 23:41:41
40372   2021-06-30 23:49:12
40373   2021-06-30 23:56:21
40374   2021-06-30 23:58:09
Name: OrderTime, Length: 40375, dtype: datetime64[ns]

In [9]:
pd.to_datetime(owi.OrderTime)[0]

Timestamp('2021-01-01 03:51:42')

In [10]:
pd.to_datetime(owi.OrderTime)[0].date()

datetime.date(2021, 1, 1)

In [11]:
str(pd.to_datetime(owi.OrderTime)[0].date())

'2021-01-01'

In [ ]:
# owi['OrderT_v1'] == owi.OrderTime.apply(lambda x: x == str(pd.to_datetime(x.OrderTime)[0].date()))

In [ ]:
# owi.OrderTime.apply(lambda x: x == str(pd.to_datetime(x.OrderTime).date()))

In [12]:
owi.OrderTime[0][0:10]

'2021-01-01'

In [13]:
owi.OrderTime.apply(lambda x: x[0:10])

0        2021-01-01
1        2021-01-01
2        2021-01-01
3        2021-01-01
4        2021-01-01
            ...    
40370    2021-06-30
40371    2021-06-30
40372    2021-06-30
40373    2021-06-30
40374    2021-06-30
Name: OrderTime, Length: 40375, dtype: object

Используем более быстрый способ, по срезу строки.

In [14]:
owi.OrderTime.apply(lambda x: x[0:10])

0        2021-01-01
1        2021-01-01
2        2021-01-01
3        2021-01-01
4        2021-01-01
            ...    
40370    2021-06-30
40371    2021-06-30
40372    2021-06-30
40373    2021-06-30
40374    2021-06-30
Name: OrderTime, Length: 40375, dtype: object

In [15]:
owi['OrderT_v2']= owi.OrderTime.apply(lambda x: x[0:10])

In [16]:
owi.head(2)

,OrderID,UserID,OrderTime,items,OrderT_v2
0,90659865,1.609460e+18,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit...",2021-01-01
1,90622054,2.499293e+18,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit...",2021-01-01


In [17]:
owi.columns

Index(['OrderID', 'UserID', 'OrderTime', 'items', 'OrderT_v2'], dtype='object')

In [18]:
owi.OrderT_v2

0        2021-01-01
1        2021-01-01
2        2021-01-01
3        2021-01-01
4        2021-01-01
            ...    
40370    2021-06-30
40371    2021-06-30
40372    2021-06-30
40373    2021-06-30
40374    2021-06-30
Name: OrderT_v2, Length: 40375, dtype: object

In [19]:
owi_grouped = owi.groupby(['OrderT_v2'])
#.agg('OrderID', sum())

In [20]:
owi_grouped.head()

,OrderID,UserID,OrderTime,items,OrderT_v2
0,90659865,1.609460e+18,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit...",2021-01-01
1,90622054,2.499293e+18,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit...",2021-01-01
2,90659952,3.945866e+17,2021-01-01 05:48:53,"[{'ItemID': '306553', 'price': 101.9, 'quantit...",2021-01-01
3,89765281,5.412765e+18,2021-01-01 08:33:41,"[{'ItemID': '979703', 'price': 99.9, 'quantity...",2021-01-01
4,90582308,6.380061e+18,2021-01-01 09:55:35,"[{'ItemID': '303765', 'price': 44.9, 'quantity...",2021-01-01
...,...,...,...,...,...
40183,93775131,9.789680e+18,2021-06-30 00:03:32,"[{'ItemID': '961330', 'price': 429.9, 'quantit...",2021-06-30
40184,93863937,8.297042e+18,2021-06-30 00:14:18,"[{'ItemID': '309885', 'price': 28.9, 'quantity...",2021-06-30
40185,93939920,2.836912e+17,2021-06-30 00:31:31,"[{'ItemID': '699444', 'price': 498.9, 'quantit...",2021-06-30
40186,93939704,3.841788e+18,2021-06-30 00:31:45,"[{'ItemID': '1052750', 'price': 17.9, 'quantit...",2021-06-30


In [21]:
maxOrders = owi_grouped['OrderID'].count().max()
maxOrders

342

In [22]:
owi_grS = owi_grouped['OrderID'].count()
owi_grS

OrderT_v2
2021-01-01     75
2021-01-02    154
2021-01-03    164
2021-01-04    212
2021-01-05    226
             ... 
2021-06-26    179
2021-06-27    154
2021-06-28    178
2021-06-29    196
2021-06-30    192
Name: OrderID, Length: 181, dtype: int64

In [23]:
type(owi_grS)

pandas.core.series.Series

In [24]:
owi_grS = owi_grouped['OrderID'].count()

In [25]:
owi_grS

OrderT_v2
2021-01-01     75
2021-01-02    154
2021-01-03    164
2021-01-04    212
2021-01-05    226
             ... 
2021-06-26    179
2021-06-27    154
2021-06-28    178
2021-06-29    196
2021-06-30    192
Name: OrderID, Length: 181, dtype: int64

In [26]:
owi_grS.sort_values()

OrderT_v2
2021-01-01     75
2021-05-02    124
2021-06-14    148
2021-05-23    149
2021-06-06    153
             ... 
2021-02-13    313
2021-01-16    318
2021-02-12    321
2021-03-06    335
2021-02-05    342
Name: OrderID, Length: 181, dtype: int64

И, находим ответ, дату, где совершенно максимальное количество покупок.

In [27]:
owi_grS.idxmax()

'2021-02-05'

Максимальное количество товаров совершено '2021-02-05'

2. Давайте посмотрим на категории товаров. Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)?

In [28]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Yandex/resources/items_with_categ.csv')
df.head()

,ItemID,Category,ParentCategory
0,1000002,Сладости для диабетиков,Сладости
1,1000046,Вода,Напитки
2,1000048,Сладости для диабетиков,Сладости
3,1000049,Вода,Напитки
4,1000061,Фрукты,Овощи и фрукты


In [29]:
owi.head(2)

,OrderID,UserID,OrderTime,items,OrderT_v2
0,90659865,1.609460e+18,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit...",2021-01-01
1,90622054,2.499293e+18,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit...",2021-01-01


In [30]:
owi['items'][0]

[{'ItemID': '413736', 'price': 36.72, 'quantity': 2},
 {'ItemID': '523171', 'price': 90.32, 'quantity': 1},
 {'ItemID': '452831', 'price': 223.11, 'quantity': 1},
 {'ItemID': '303673', 'price': 59.9, 'quantity': 7},
 {'ItemID': '318193', 'price': 319.91, 'quantity': 2},
 {'ItemID': '345578', 'price': 159.92, 'quantity': 1},
 {'ItemID': '316140', 'price': 135.92, 'quantity': 2},
 {'ItemID': '444061', 'price': 291.12, 'quantity': 2},
 {'ItemID': '500982', 'price': 36.72, 'quantity': 3},
 {'ItemID': '303682', 'price': 63.26, 'quantity': 2},
 {'ItemID': '308467', 'price': 35.11, 'quantity': 3},
 {'ItemID': '312673', 'price': 289.9, 'quantity': 1},
 {'ItemID': '429288', 'price': 179.9, 'quantity': 1},
 {'ItemID': '306876', 'price': 29.9, 'quantity': 1},
 {'ItemID': '306843', 'price': 75.12, 'quantity': 1},
 {'ItemID': '521581', 'price': 79.92, 'quantity': 1},
 {'ItemID': '303669', 'price': 116.87, 'quantity': 1},
 {'ItemID': '313776', 'price': 99.9, 'quantity': 1},
 {'ItemID': '521656', 'pr

In [31]:
owi['items'][0][0]

{'ItemID': '413736', 'price': 36.72, 'quantity': 2}

In [32]:
owi['items'][0][0]['price']

36.72

In [33]:
owi['items'][0][0]

{'ItemID': '413736', 'price': 36.72, 'quantity': 2}

У нас в первой таблице находится ItemID в owi['items'] в качестве ключа.
Также у нас там есть price (цена) и quantity (количество). Нас интересует GVM, суммарная выручка. Например, по Сладостям.

Нам нужно из таблицы owi сделать таблицу, где каждому ItemID сделать свою GVM. Потом можно смержить две таблицы, и найти уже по старшей категории GVM.

Работаем с owi['items']

У нас в каждой колонке есть словарь, откуда нужно взять значение по ключу ItemID, перенести в другую таблицу, перемножить price*quantity и перенести в другую колонку. Таким образом мы создадим новый датафрейм, где у нас будут колонки ItemID и GVM_ItemID

In [34]:
owi['items'][0]

[{'ItemID': '413736', 'price': 36.72, 'quantity': 2},
 {'ItemID': '523171', 'price': 90.32, 'quantity': 1},
 {'ItemID': '452831', 'price': 223.11, 'quantity': 1},
 {'ItemID': '303673', 'price': 59.9, 'quantity': 7},
 {'ItemID': '318193', 'price': 319.91, 'quantity': 2},
 {'ItemID': '345578', 'price': 159.92, 'quantity': 1},
 {'ItemID': '316140', 'price': 135.92, 'quantity': 2},
 {'ItemID': '444061', 'price': 291.12, 'quantity': 2},
 {'ItemID': '500982', 'price': 36.72, 'quantity': 3},
 {'ItemID': '303682', 'price': 63.26, 'quantity': 2},
 {'ItemID': '308467', 'price': 35.11, 'quantity': 3},
 {'ItemID': '312673', 'price': 289.9, 'quantity': 1},
 {'ItemID': '429288', 'price': 179.9, 'quantity': 1},
 {'ItemID': '306876', 'price': 29.9, 'quantity': 1},
 {'ItemID': '306843', 'price': 75.12, 'quantity': 1},
 {'ItemID': '521581', 'price': 79.92, 'quantity': 1},
 {'ItemID': '303669', 'price': 116.87, 'quantity': 1},
 {'ItemID': '313776', 'price': 99.9, 'quantity': 1},
 {'ItemID': '521656', 'pr

In [35]:
owi['items'][0][0]['ItemID']

'413736'

Покажу на примере первого заказа (нулевая строка в датафрейме owi), то, что я хочу вычленить из таблицы. Фактически нас интресует 'ItemID' и соответствующий cost = price * quantity (стоимость = цена * количество)

In [36]:
owi['items'][0][1]['ItemID']

'523171'

In [37]:
owi['items'][0][1]['price'] * owi['items'][0][1]['quantity']

90.32

Теперь надо проитерироваться по series - столбцу owi['items'] и просуммировать по всем уникальным номерам items произведение price * quantity.

In [38]:
list_ofItemIDitems  = owi['items'].to_list()

In [39]:
list_ofItemIDitems [0]

[{'ItemID': '413736', 'price': 36.72, 'quantity': 2},
 {'ItemID': '523171', 'price': 90.32, 'quantity': 1},
 {'ItemID': '452831', 'price': 223.11, 'quantity': 1},
 {'ItemID': '303673', 'price': 59.9, 'quantity': 7},
 {'ItemID': '318193', 'price': 319.91, 'quantity': 2},
 {'ItemID': '345578', 'price': 159.92, 'quantity': 1},
 {'ItemID': '316140', 'price': 135.92, 'quantity': 2},
 {'ItemID': '444061', 'price': 291.12, 'quantity': 2},
 {'ItemID': '500982', 'price': 36.72, 'quantity': 3},
 {'ItemID': '303682', 'price': 63.26, 'quantity': 2},
 {'ItemID': '308467', 'price': 35.11, 'quantity': 3},
 {'ItemID': '312673', 'price': 289.9, 'quantity': 1},
 {'ItemID': '429288', 'price': 179.9, 'quantity': 1},
 {'ItemID': '306876', 'price': 29.9, 'quantity': 1},
 {'ItemID': '306843', 'price': 75.12, 'quantity': 1},
 {'ItemID': '521581', 'price': 79.92, 'quantity': 1},
 {'ItemID': '303669', 'price': 116.87, 'quantity': 1},
 {'ItemID': '313776', 'price': 99.9, 'quantity': 1},
 {'ItemID': '521656', 'pr

In [40]:
len(list_ofItemIDitems)
#Естественно равно количеству строк в первоначальном датасете

40375

Пробежимся по всем json

In [41]:
itemIDs_cost = list()
for el1 in list_ofItemIDitems:
  for el2 in el1:
    v1 = el2['ItemID']
    v2 = el2['price'] * el2['quantity']
    itemIDs_cost.append([v1, v2])
    #ItemID_cost['ItemID'] = el2['ItemID']
    #ItemID_cost['cost'] = el2['price'] * el2['quantity']
    #print(el2['ItemID'], el2['price'] * el2['quantity'])

In [42]:
ItemID_cost = pd.DataFrame(itemIDs_cost)

In [43]:
ItemID_cost.columns = ['ItemID','cost']

Мы получили датафрейм, где расписаны все единицы товара (ItemID) по стоимости.

In [44]:
ItemID_cost.head()

,ItemID,cost
0,413736,73.44
1,523171,90.32
2,452831,223.11
3,303673,419.30
4,318193,639.82


Сгруппируем по ItemID и просуммируем по cost

In [45]:
ItemID_cost.groupby('ItemID').sum().reset_index()

,ItemID,cost
0,1000002,199.30
1,1000046,3810.38
2,1000048,191.40
3,1000049,898.52
4,1000061,341.81
...,...,...
35025,999942,294.60
35026,999964,159.90
35027,999965,665.50
35028,999970,3436.81


Запишем полученный датафрейм в переменную.

In [46]:
ItemIDcost_grouped = ItemID_cost.groupby('ItemID').sum().reset_index()
ItemIDcost_grouped.head()

,ItemID,cost
0,1000002,199.30
1,1000046,3810.38
2,1000048,191.40
3,1000049,898.52
4,1000061,341.81


Ещё у нас есть такой датафрейм

In [47]:
df.head(2)

,ItemID,Category,ParentCategory
0,1000002,Сладости для диабетиков,Сладости
1,1000046,Вода,Напитки


Теперь мы **смержим** оба датафрейма по 'ItemID', просуммируем cost по 'ParentCategory'.

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35030 entries, 0 to 35029
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ItemID          35030 non-null  int64 
 1   Category        35030 non-null  object
 2   ParentCategory  35030 non-null  object
dtypes: int64(1), object(2)
memory usage: 821.1+ KB


In [49]:
[df.ParentCategory.nunique(), df.ParentCategory.unique()]

[17, array(['Сладости', 'Напитки', 'Овощи и фрукты',
        'Красота, гигиена, здоровье', 'Для праздника',
        'Кухонные принадлежности', 'Бакалея', 'Молочная продукция',
        'Мясо, птица, рыба', 'Мучные изделия', 'Зоотовары',
        'Для дома и сада', 'Бытовая химия', 'Для детей',
        'Системы нагревания', 'Текстиль', 'Для школы'], dtype=object)]

In [50]:
df.sort_values('ParentCategory', ascending = True)

,ItemID,Category,ParentCategory
17514,452557,Диабетическая бакалея,Бакалея
4651,1051082,Готовые блюда охлажденные,Бакалея
26679,740128,"Горчица, хрен",Бакалея
26670,727606,Соусы,Бакалея
26650,725319,"Масло, маргарин, спред",Бакалея
...,...,...,...
32975,986355,Постельное белье,Текстиль
417,1003136,Текстиль,Текстиль
421,1003184,Текстиль,Текстиль
16874,445317,"Носки, гольфы",Текстиль


Используем метод merge()

Обычно в методе merge используется левое соединение.
Представим ситуацию, что в первом датасете ItemID_cost у нас могут быть разнообразные ItemID, даже те, которые не присутствуют в датасете df. Поэтому, в методе df надо первым (левым) аргументом сделать датасет df

In [51]:
ItemIDcost_grouped.head()

,ItemID,cost
0,1000002,199.30
1,1000046,3810.38
2,1000048,191.40
3,1000049,898.52
4,1000061,341.81


In [52]:
df.dtypes

ItemID             int64
Category          object
ParentCategory    object
dtype: object

In [53]:
ItemIDcost_grouped.dtypes

ItemID     object
cost      float64
dtype: object

Приведём ItemID к единому типу в обоих датафреймах, поскольку у нас df дан изначально, значит в ItemIDcost_grouped изменим тип колонки object на int64

In [54]:
ItemIDcost_grouped['ItemID'] = ItemIDcost_grouped['ItemID'].astype('int64')

In [55]:
ItemIDcost_grouped.dtypes

ItemID      int64
cost      float64
dtype: object

Теперь можно безболезненно мерджить таблицы по однотипной колонки с одинаковым названием.

In [56]:
PC_GVM  = df.merge(ItemIDcost_grouped)
PC_GVM

,ItemID,Category,ParentCategory,cost
0,1000002,Сладости для диабетиков,Сладости,199.30
1,1000046,Вода,Напитки,3810.38
2,1000048,Сладости для диабетиков,Сладости,191.40
3,1000049,Вода,Напитки,898.52
4,1000061,Фрукты,Овощи и фрукты,341.81
...,...,...,...,...
35025,999942,Шоколадные батончики,Сладости,294.60
35026,999964,Зелень и салаты,Овощи и фрукты,159.90
35027,999965,"Пироги, сдоба, кексы, рулеты",Мучные изделия,665.50
35028,999970,"Прокладки, тампоны","Красота, гигиена, здоровье",3436.81


In [57]:
PC_GVM.dtypes

ItemID              int64
Category           object
ParentCategory     object
cost              float64
dtype: object

Отсортируем по 'ParentCategory' и проссумируем по cost.

In [70]:
PC_GVM = PC_GVM.groupby('ParentCategory', as_index=True)[['ParentCategory','cost']].agg({'cost': 'sum'})

In [73]:
PC_GVM.reset_index().sort_values('cost', ascending = False).head(3)

,ParentCategory,cost
11,"Мясо, птица, рыба",32268493.46
9,Молочная продукция,27702995.43
13,Овощи и фрукты,27607486.90


**Можно это делать и с помощью nlargest()...** для тех, кто не знает как. :)

In [76]:
top3_PC = PC_GVM.reset_index().sort_values('cost', ascending = False)['ParentCategory'].head(3).tolist()

**Самые продаваемые по GVM родительские категории:**

In [77]:
top3_PC

['Мясо, птица, рыба', 'Молочная продукция', 'Овощи и фрукты']


**3. Представьте, что мы хотим запустить какую-то акцию для клиентов или предложить какую-то бонусную программу. Какую акцию было бы целесообразно провести и на каких клиентов её ориентировать? Составьте какой-то список клиентов, которым вы бы предложили участие в акции/программе.**

Можно найти клиентов, которые совершили максимальное количество покупок. Найти клиентов, которые совершили максимальную по стоимости закупку. Для данных клиентов я бы изучил какие товары они чаще покупают. То есть внутри их покупок выделил те, которые они чаще покупают. И уменьшил бы стоимость товара.

Данная задача делается по аналогии с предыдущей, нужно смотреть только не по 'ItemID', а по 'UserID' смердженную таблицу.

**4. Представьте, что для повышения GMV мы хотим внедрить на сайте систему рекомендаций и предлагать пользователю какие-то товары по мере сбора корзины. Что бы вы порекомендовали пользователю X?**

Я полагаю, надо смотреть в реальном времени, какой категории товара больше всего закупается пользователь X и по категории товара, внутри более крупной категории выбрал бы более подходящий. Ещё есть такой способ рекомендации, с каким товаром чаще покупают другой товар. Есть такая ситуация. Ещё можно посмотреть в какое время чаще берут какие товары и посоветовать именно этот товар. Защитил в OTUS диплом по рекомендательным системам и сейчас на курсе у Карпова по hard ml совершенствуюсь в ранжировании и матчинге (выводить топчик) и в ценообразовании.